# <center> Music Recommender System

We draw inspiration from Spotify's "Discover Weekly" service and aim to recommend a list of songs to each user according to his preference.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import random
from scipy.stats import norm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time

## Import Data 

In [3]:
df = pd.read_csv("smaller_songs_dataset.csv")
df.head()

,user_id,item_id,listening_time_in_scale_0-5,title,duration,loudness,tempo,key,event_type
0,196,242,4.06,Atmosphere Station,191.81669,-38.525,90.084,9,"""Listen it frequently""""View Song Information""""..."
1,63,242,3.24,Atmosphere Station,191.81669,-38.525,90.084,9,"""Listen it frequently""""Like""""View Song Informa..."
2,154,242,4.34,Atmosphere Station,191.81669,-38.525,90.084,9,"""Listen it frequently""""Like""""View Song Informa..."
3,34,242,3.57,Atmosphere Station,191.81669,-38.525,90.084,9,"""Have listened it completely""""View Song Infor..."
4,35,242,3.09,Atmosphere Station,191.81669,-38.525,90.084,9,"""Have listened it completely""""View Song Infor..."


In [4]:
df['event_type'].value_counts()

"Listen it frequently""Download"                                                               286
"Have listened  it completely""Like""View Song Information""Download"                          283
"Listen it frequently""Add it to your music"                                                   282
"Have listened  it completely""View Song Information""Add it to your music"                    279
"Skip it""Not recommend it anymore"                                                            278
"Listen it frequently""Like""View Song Information"                                            275
"Have listened  it completely""Download"                                                       273
"Skip it frequently""Not recommend it anymore"                                                 273
"Listen it frequently""Like""Download"                                                         272
"Have listened  it completely""Like""Add it to your music"                                     270
"Have list

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9409 entries, 0 to 9408
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   user_id                      9409 non-null   int64  
 1   item_id                      9409 non-null   int64  
 2   listening_time_in_scale_0-5  9409 non-null   float64
 3   title                        9409 non-null   object 
 4   duration                     9409 non-null   float64
 5   loudness                     9409 non-null   float64
 6   tempo                        9409 non-null   float64
 7   key                          9409 non-null   int64  
 8   event_type                   9409 non-null   object 
dtypes: float64(4), int64(3), object(2)
memory usage: 661.7+ KB


In [6]:
df['user_id'].value_counts()

1      272
13     191
92     181
94     160
59     159
      ... 
61       6
147      5
111      5
36       5
88       3
Name: user_id, Length: 200, dtype: int64

In [7]:
df['item_id'].value_counts()

50     118
258    116
100    105
294    102
286    100
      ... 
146      2
296      2
74       1
75       1
119      1
Name: item_id, Length: 300, dtype: int64

## Data Preprocessing 

Define strength for each event in 'event_type'

Quantify implicit event feedback 

In [8]:
# event --> strength
event_type_strength = {
    'Listen it frequently': 3.0,
    'Have listened  it completely': 1.0,
    'Like': 1.0,
    'View Song Information': 1.0,
    'Download': 3.0,
    'Add it to your music': 2.0,
    'Skip it':  -1.0, 
    'Skip it frequently': -3.0,
    'Not recommend it anymore': -2.0}

In [9]:
# extract event into a list from a string
df['event_type'] = df['event_type'].apply(lambda x: [e for e in x.split("\"") if len(e)])

# add a column event_strength
df['event_strength'] = df['event_type'].apply(lambda x: np.sum([event_type_strength[e] for e in x]))

In [10]:
# stats on event strength
df['event_strength'].value_counts()

 6.0     1328
 4.0     1323
 5.0     1322
 7.0     1313
 3.0      787
 8.0      743
-3.0      526
 9.0      521
 2.0      501
-5.0      273
-1.0      265
 1.0      257
 10.0     250
Name: event_strength, dtype: int64

In [11]:
df.head()

,user_id,item_id,listening_time_in_scale_0-5,title,duration,loudness,tempo,key,event_type,event_strength
0,196,242,4.06,Atmosphere Station,191.81669,-38.525,90.084,9,"[Listen it frequently, View Song Information, ...",7.0
1,63,242,3.24,Atmosphere Station,191.81669,-38.525,90.084,9,"[Listen it frequently, Like, View Song Informa...",5.0
2,154,242,4.34,Atmosphere Station,191.81669,-38.525,90.084,9,"[Listen it frequently, Like, View Song Informa...",7.0
3,34,242,3.57,Atmosphere Station,191.81669,-38.525,90.084,9,"[Have listened it completely, View Song Infor...",5.0
4,35,242,3.09,Atmosphere Station,191.81669,-38.525,90.084,9,"[Have listened it completely, View Song Infor...",4.0


## Spotify environment 

In [12]:
def random_argmax(rng, list_):
    """ return a random element among max when multiple max exists."""
    return rng.choice(np.argwhere(list_ == list_.max()).flatten())

In [13]:
np.set_printoptions(precision=2)

In [14]:
class music_rec_env:
    """ A rating environment that simulates Spotify.
        User and items are represented by points in R^3 modelling song style (loudness, tempo, key) 
        User interest for a given item is modeled by 3 parts:
        1. dot product of user,item embedding vector (cosine similarity, style coherence)
        2. user listening time estimation
        3. user event estimation (event_strength)
        action: Recommend a song list for a given user among those he has never listened to before
    """
    
    def __init__(self, data, seed=None,
                 internal_embedding_size=3,
                 displayed_users_embedding_size=2,
                 displayed_items_embedding_size=2,
                 noise_size=2):
        self.raw_data = data  # pandas dataframe with 'user_id', 'item_id' columns
        self.nb_users = len(df['user_id'].value_counts())
        self.nb_items = len(df['item_id'].value_counts())
        self.internal_embedding_size = internal_embedding_size
        self.displayed_users_embedding_size = displayed_users_embedding_size
        self.displayed_items_embedding_size = displayed_items_embedding_size
        self.noise_size = noise_size
        self._rng = np.random.RandomState(seed)
        
        self.action_size = self.nb_items
        self.sampling_limit = self.nb_users * self.nb_items
        self.users_embedding = None
        self.items_embedding = None
        self.user_item_history = None
        self.done = False

    def step(self, action_list):
        # check if behind done
        if self.done:
            print("Calling step after it returns done=True.\n"
                  "Please reset the environment.")

        # compute potential rewards
        potential_rewards = [self._get_user_item_rating(self.current_user, i) 
                             for i in np.argwhere(self.user_item_history[self.current_user, :] == 0).flatten()]
        
        # compute optimal reward
        optimal_returns = np.array(sorted(potential_rewards, reverse = True))[:len(action_list)]

        # map actions to items
        self.recommended_items = [np.argwhere(self.user_item_history[self.current_user, :] == 0)[action][0] for action in action_list]

        # mark items as rated
        self.user_item_history[self.current_user, self.recommended_items] = 1

        # compute reward R_t
        self.current_ratings = [self._get_user_item_rating(self.current_user, rec_item) for rec_item in self.recommended_items]
        self.rewards = self.current_ratings
        
        # check if done
        if self.user_item_history.sum() == self.sampling_limit:
            self.done = True

        # compute next state S_{t+1}
        self._next_state()

        # update action space t+1
        self.action_size = len(self.available_items)

        return self.rewards, self.state, self.done, optimal_returns

    def reset(self, data=None, seed=None):
        
        if data:
            self.raw_data = data
            
        if seed:
            self._rng = np.random.RandomState(seed)
 
        # create item embedding matrix (song style features) 
        song_df = self.raw_data[['item_id','loudness','tempo','key']].drop_duplicates().sort_values(by = 'item_id')
        style_raw_data = np.vstack((song_df['loudness'], song_df['tempo'],song_df['key'])).T
        min_max_scaler = MinMaxScaler()
        self.items_embedding = min_max_scaler.fit_transform(style_raw_data)
        
        # create user embedding matrix (song style preference features)
        self.users_embedding = np.array([])
        for user in np.array(sorted(self.raw_data['user_id'].unique())):
            user_raw_data = self.raw_data[self.raw_data['user_id']==user]
            tmp = user_raw_data[['loudness','tempo','key']].mul(user_raw_data['listening_time_in_scale_0-5'], axis=0)
            tmp = tmp.div(user_raw_data['listening_time_in_scale_0-5'].sum())
            user_unscaled_data = np.array(tmp.mean(axis=0))
            self.users_embedding = np.append(self.users_embedding, user_unscaled_data)    
        self.users_embedding = min_max_scaler.fit_transform(self.users_embedding.reshape(self.raw_data['user_id'].nunique(),3))    

        # initialize history matrix, recommend history + listening history
        # listening/reocmmend history = 1
        self.user_item_history = np.zeros((self.nb_users, self.nb_items))
        v_func = np.vectorize(self._get_listening_history)
        v_func(a=np.array(self.raw_data['user_id']), b=np.array(self.raw_data['item_id']))
        
        self.done = False

        self._next_state()
        return self.state
     
    def _get_listening_history(self, a, b):
        # record listening history in raw_data
        # a: user_id, b: item_id, M: ndarray
        self.user_item_history[a-1,b-1] = 1  # both id start from 1

    def _get_user_item_rating(self, user, item): 
        # score = style_matching_score + estimated_time_bonus + estimated_event_bonus
        # all 3 parts range from 0 to 1

        # compute style_matching_score
        style_matching_score = self.users_embedding[user].dot(self.items_embedding[item])
        
        user_raw_data = self.raw_data[self.raw_data['user_id']==user]
        songs_similarity = self.items_embedding[np.array(user_raw_data['item_id'])-1,:].dot(self.items_embedding[item])
        
        # compute estimated_time_bonus   
        unscaled_estimated_time_bonuses = user_raw_data['listening_time_in_scale_0-5'].mul(songs_similarity)
        scaled_estimated_time_bonuses = unscaled_estimated_time_bonuses.div(np.sum(songs_similarity))
        estimated_time_bonus = (scaled_estimated_time_bonuses.sum() - user_raw_data['listening_time_in_scale_0-5'].min())/ \
                               (user_raw_data['listening_time_in_scale_0-5'].max() - user_raw_data['listening_time_in_scale_0-5'].min())
        
        # compute estimated_event_bonus
        unscaled_estimated_event_bonuses = user_raw_data['event_strength'].mul(songs_similarity)
        scaled_estimated_event_bonuses = unscaled_estimated_event_bonuses.div(np.sum(songs_similarity))
        estimated_event_bonus = (scaled_estimated_event_bonuses.sum() - user_raw_data['event_strength'].min())/ \
                                (user_raw_data['event_strength'].max() - user_raw_data['event_strength'].min())
        
      #  rand_engine = np.random.RandomState(0)    
        unexplained_changes = np.random.normal(loc=0.0, scale=0.2)
        score = style_matching_score + estimated_time_bonus + estimated_event_bonus + unexplained_changes
        return score

    def _get_variables(self, user, item):
        user_embedding = self.users_embedding[user]
        item_embedding = self.items_embedding[item]
        if self.displayed_users_embedding_size + self.displayed_items_embedding_size > 0:
            variables = np.array([user_embedding[:self.displayed_users_embedding_size],
                                  item_embedding[:self.displayed_items_embedding_size]])

            if self.noise_size > 0:
                noise = self._rng.normal(loc=np.zeros(self.noise_size),
                                         scale=np.ones(self.noise_size)/2,
                                         size=self.noise_size)
                variables = np.append(variables, noise)

            return variables

    def _get_new_user(self):
        has_picked = False
        user = self._rng.randint(0, self.nb_users)
        # get a new user who hasn't tried all the songs
        while np.sum(self.user_item_history[user, :]) == self.nb_items:
            user = self._rng.randint(0, self.nb_users)
        return user    

    def _next_state(self):
        # Pick a user
        if self.user_item_history.sum() < self.sampling_limit: 
            self.current_user = self._get_new_user()
        else:
            self.current_user = None

        # list available items
        self.available_items = np.argwhere(self.user_item_history[self.current_user, :] == 0)

        self.state = list()
        for i in self.available_items:
            item = i[0]   
            # compute variables (part of the user, item embedding (with noise))
            variables = self._get_variables(self.current_user, item)
            self.state.append([self.current_user+1, item+1, variables])


In [15]:
env =  music_rec_env(data = df, seed = 2020)

In [16]:
env.reset(seed=2020)[:10]  # user, available_item, user_item_feature with noise

[[97, 2, array([ 0.94,  0.04,  0.79,  0.41,  0.18, -0.51])],
 [97, 3, array([ 0.94,  0.04,  0.8 ,  0.49,  0.05, -0.61])],
 [97, 4, array([ 0.94,  0.04,  0.93,  0.54,  0.99, -0.04])],
 [97, 5, array([ 0.94,  0.04,  0.8 ,  0.61, -0.42,  0.09])],
 [97, 6, array([ 0.94,  0.04,  0.58,  0.46, -0.13,  0.91])],
 [97, 8, array([ 0.94,  0.04,  0.68,  0.36, -0.2 , -0.01])],
 [97, 9, array([ 0.94,  0.04,  0.87,  0.47,  0.03, -1.32])],
 [97, 10, array([ 0.94,  0.04,  0.78,  0.61, -0.62, -0.79])],
 [97, 11, array([ 0.94,  0.04,  0.84,  0.49, -0.15, -0.56])],
 [97, 12, array([ 0.94,  0.04,  0.87,  0.54,  0.4 , -0.29])]]

In [17]:
np.array(sorted(df[df['user_id']==97]['item_id']))[:10]

array([ 1,  7, 23, 28, 32, 50, 69, 79, 82, 83])

User 97 is connecting to the platform.

Recommend him three songs among all available songs.

For each user-item pair, observe also a vector of features that depends on the user, the song and/or some context(like occasion, weather, user's emotion, etc).

In [18]:
reward, next_state, done, optimal_return = env.step(np.arange(3)) # pick the first 3 available items in the list 
print('reward: ', reward)
print('best reward:',optimal_return)

reward:  [2.0338785738452354, 2.017059580487061, 2.033373336338571]
best reward: [2.63 2.6  2.5 ]


In [19]:
next_state[:10]

[[8, 1, array([ 0.95,  0.05,  0.78,  0.5 , -0.25, -0.23])],
 [8, 2, array([ 0.95,  0.05,  0.79,  0.41, -0.02, -0.27])],
 [8, 3, array([ 0.95,  0.05,  0.8 ,  0.49,  0.59, -0.96])],
 [8, 4, array([ 0.95,  0.05,  0.93,  0.54, -0.27,  0.34])],
 [8, 5, array([ 0.95,  0.05,  0.8 ,  0.61, -0.77,  0.56])],
 [8, 6, array([ 0.95,  0.05,  0.58,  0.46,  0.14, -0.34])],
 [8, 8, array([ 0.95,  0.05,  0.68,  0.36, -0.65,  0.7 ])],
 [8, 9, array([ 0.95,  0.05,  0.87,  0.47, -0.04,  0.52])],
 [8, 10, array([ 0.95,  0.05,  0.78,  0.61,  0.53, -0.3 ])],
 [8, 12, array([ 0.95,  0.05,  0.87,  0.54,  0.63, -0.64])]]

In [20]:
np.array(sorted(df[df['user_id']==54]['item_id']))[:10]

array([  1,   7,  24,  25,  50, 100, 106, 117, 118, 121])

In [21]:
env.reset(seed=2000)[:10]
reward, next_state, done, optimal_return = env.step(np.arange(3)) # pick the first 3 available items in the list 
print('reward: ', reward)
print('best reward:',optimal_return)

reward:  [2.013929537409577, 1.692477773465048, 1.692417186802118]
best reward: [2.4  2.31 2.27]


## Agent

### Random Agent

In [22]:
class Random:
    """ Random agent. """
    def __init__(self, nb_arms, seed=None):
        self._nb_arms = nb_arms
        self._rng = np.random.RandomState(seed)
        
    def act(self, context):
        actions = self._rng.randint(len(context), size = self._nb_arms)  
        return actions
    
    def update(self, context, action, reward):
        pass

In [23]:
random_agent = Random(nb_arms = 3, seed=0)

### ALS 

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [25]:
from pyspark.ml.recommendation import ALS
from scipy.sparse import *

In [26]:
class ALS_matrix_factorization:
    """ ALS agent. """
    def __init__(self, ratings, nb_arms ,seed=None):
        self._ratings = ratings
        self._n_user = ratings.shape[0]
        self._n_item = ratings.shape[1]
        self._nb_arms = nb_arms
        self._policy = ALS(maxIter=10, regParam=1, rank=10, implicitPrefs = False, userCol="user_id", itemCol="item_id", ratingCol="rating",coldStartStrategy="drop")
        self._rng = np.random.RandomState(seed)
        
    def act(self, context, new_rating):
        user = context[0][0]  # current user id
        self._ratings = new_rating
        ratings = copy(self._ratings)
        available_items = [l[1] for l in context]
        
        df = self.convert_to_df(ratings)
        test_df = df[(df['user_id'] == user) & (df['item_id'].isin(available_items))]  # forecast target
        train_df = df[df['rating'] != 0]
        train_df = spark.createDataFrame(train_df)
        test_df = spark.createDataFrame(test_df)

        model = self._policy.fit(train_df)
        predictions = model.transform(test_df) # predicted ratings for all available items (current user)
       

        predictions = predictions.toPandas()
        pred_ratings = predictions['prediction']

        
        pred_ratings = np.array([-pred_ratings.at[i] for i in range(pred_ratings.shape[0])])
        
     
        threshold = min(self._nb_arms, len(pred_ratings))
        
        actions = np.argpartition(pred_ratings, threshold)[:threshold]  # sorting in ascending order 
  
        return actions
    
    def convert_to_df(self, ratings, name='rating'):
        # convert a ndarray to pd.Dataframe
        n_user, n_item = self._n_user, self._n_item
        arr = np.array([np.repeat(np.arange(n_user), n_item),np.tile(np.arange(n_item), n_user),ratings.flatten()]).T
        df = pd.DataFrame(arr,columns=['user_id','item_id',name], dtype = int)
        return df
        
    def update(self, context, actions, rewards):
        last_user = context[0][0]
        last_reco_items = np.array([context[a][1] for a in actions]) 
        for item, rwd in zip(last_reco_items, rewards):
            self._ratings[last_user, item] = rwd  # reward = rating 

## Experiment

In [27]:
def run_exp(agent, env, nb_steps, nb_arms, env_seed, ratings = None):
    rewards_list = np.zeros((nb_steps, nb_arms))
    regrets_list = np.zeros((nb_steps, nb_arms))
    actions_list = np.zeros((nb_steps, nb_arms))
    context = env.reset(data = None, seed = env_seed)
    
    has_rating_history = False
    try:
        ratings.shape
        rating_matrix = ratings 
        initial_rating_matrix = ratings
        has_rating_history = True
    except:
        rating_matrix = np.zeros((env.nb_users, env.nb_items))
        initial_rating_matrix = np.zeros((env.nb_users, env.nb_items))

    for i in range(nb_steps):
        # Select action from agent policy.
        
        if has_rating_history: # agent needs history data (ex. ALS)
            actions = agent.act(context, rating_matrix)
        else:
            actions = agent.act(context)
        
        # Play action in the environment and get reward.
        rewards, next_context, done, optimal_return = env.step(actions)
        
        # Update history
        user = context[0][0]
        items = np.array([context[a][1] for a in actions], dtype = int)  # a list of [user_id, item_id, ]

        rating = rewards
        rating_matrix[user-1, items-1] = rating
        
        # Update agent.
        agent.update(context, actions, rewards)
        context = next_context
        
        # Save history.
        rewards_list[i] = rewards
        actions_list[i] = actions
        regrets_list[i] = optimal_return - rewards

    reward = np.sum(rewards_list, axis = 0)
    regret = np.sum(regrets_list, axis = 0)
 
    return {'reward': reward, 
            'regret': regret,
            'rewards': rewards_list,
            'regrets': regrets_list,
            'actions': actions_list,
            'cum_rewards': np.cumsum(rewards_list, axis = 0), 
            'cum_regrets': np.cumsum(regrets_list, axis = 0),
            'rating_matrix': rating_matrix-initial_rating_matrix
            }


In [28]:
res = run_exp(random_agent, env, nb_steps=20, nb_arms=3, env_seed=0)

In [29]:
res

{'reward': array([41.01, 39.14, 40.33]),
 'regret': array([12.18, 12.76, 10.73]),
 'rewards': array([[2.44, 1.73, 2.16],
        [1.71, 2.73, 2.38],
        [2.27, 2.61, 1.72],
        [2.28, 1.67, 2.56],
        [2.23, 2.43, 2.07],
        [2.1 , 2.23, 2.2 ],
        [2.29, 1.6 , 2.19],
        [1.87, 1.4 , 1.86],
        [2.37, 2.04, 2.46],
        [1.59, 1.84, 1.75],
        [2.19, 2.16, 1.45],
        [1.96, 2.06, 1.58],
        [2.28, 2.13, 2.57],
        [1.98, 1.69, 1.66],
        [2.08, 1.83, 1.97],
        [1.97, 1.86, 2.02],
        [1.9 , 1.8 , 2.13],
        [1.5 , 1.51, 1.9 ],
        [1.74, 2.07, 1.98],
        [2.27, 1.75, 1.72]]),
 'regrets': array([[ 0.35,  1.04,  0.59],
        [ 0.89, -0.25,  0.08],
        [ 1.  ,  0.55,  1.41],
        [ 0.34,  0.82, -0.08],
        [ 0.2 , -0.06,  0.29],
        [ 0.82,  0.67,  0.6 ],
        [ 0.34,  0.98,  0.35],
        [ 0.35,  0.77,  0.31],
        [ 0.4 ,  0.62,  0.15],
        [ 1.2 ,  0.92,  0.87],
        [ 0.43,  0.38,  

In [30]:
res = run_exp(random_agent, env, nb_steps=500, nb_arms=3, env_seed=2)

In [31]:
ALS_agent = ALS_matrix_factorization(ratings = res['rating_matrix'], nb_arms = 3, seed=15)

In [33]:
for a,b in zip(np.array(df['user_id']),np.array(df['item_id'])):
    res['rating_matrix'][a-1][b-1] = np.random.normal(loc=1.5, scale=0.1)
    
res_ALS = run_exp(ALS_agent, env, nb_steps=20, nb_arms=3, env_seed=7, ratings = res['rating_matrix'])
res_ALS

{'reward': array([38.87, 40.6 , 39.7 ]),
 'regret': array([14.6 , 11.33, 11.63]),
 'rewards': array([[1.82, 2.17, 2.08],
        [1.82, 2.31, 1.73],
        [2.06, 2.32, 2.17],
        [1.43, 1.42, 2.24],
        [2.15, 1.91, 1.29],
        [2.16, 2.25, 2.04],
        [2.04, 2.  , 1.44],
        [1.84, 1.96, 2.39],
        [1.74, 2.23, 2.3 ],
        [2.28, 1.98, 2.03],
        [2.11, 1.95, 1.69],
        [2.31, 2.21, 1.73],
        [1.46, 2.01, 1.64],
        [2.44, 1.94, 2.31],
        [1.89, 1.57, 2.27],
        [1.41, 2.11, 1.71],
        [2.08, 1.97, 2.37],
        [2.08, 2.07, 2.36],
        [1.95, 2.26, 2.01],
        [1.79, 1.96, 1.92]]),
 'regrets': array([[0.61, 0.24, 0.33],
        [0.75, 0.09, 0.63],
        [0.48, 0.15, 0.3 ],
        [1.3 , 1.2 , 0.37],
        [0.26, 0.5 , 1.11],
        [0.6 , 0.43, 0.59],
        [0.55, 0.57, 1.09],
        [0.95, 0.65, 0.22],
        [1.59, 1.1 , 0.97],
        [0.5 , 0.78, 0.72],
        [0.39, 0.5 , 0.73],
        [0.31, 0.33, 0.78]

## Draw empirical regret and confidence bound

In [33]:
def draw_regret_bound(agent_name, nb_arms=3, nb_exp=10, nb_steps=10):
    '''
    Repeat environment-agent interations and draw regret bound for a given agent
    '''    
    regret = np.zeros(nb_exp)
    regrets = np.zeros((nb_exp, nb_steps))
    for i in range(nb_exp):
        env = music_rec_env(data = df, seed = i)
        agent = Random(nb_arms, seed=i)
        if agent_name == 'random':
            agent = Random(nb_arms, seed=i)
            exp = run_exp(agent, env, nb_steps, nb_arms, env_seed=2*i)
        if agent_name == 'ALS':
            agent = Random(nb_arms, seed=i)
            exp1 = run_exp(agent, env, 400, nb_arms, env_seed=2*i)
            for a,b in zip(np.array(df['user_id']),np.array(df['item_id'])):
                exp1['rating_matrix'][a-1][b-1] = np.random.normal(loc=1.5, scale=0.1)
            ALS_agent = ALS_matrix_factorization(ratings = exp1['rating_matrix'], nb_arms = 3, seed=i)
            exp = run_exp(ALS_agent, env, nb_steps, nb_arms, env_seed=2*i, ratings = exp1['rating_matrix'])
       
        regret[i] = np.sum(exp['regret'])
        regrets[i] = np.sum(exp['cum_regrets'], axis = 1) 
        
    plt.plot(regrets.mean(axis=0), color='blue')
    plt.plot(np.quantile(regrets, 0.05,axis=0), color='grey', alpha=0.5)
    plt.plot(np.quantile(regrets, 0.95,axis=0), color='grey', alpha=0.5)
    plt.title('Mean regret: {:.2f}'.format(regret.mean()))
    plt.xlabel('steps')
    plt.ylabel('regret')
    plt.show()    

In [34]:
t1 = time.perf_counter()
draw_regret_bound('random', nb_arms=3, nb_exp=4, nb_steps=20)
t2 = time.perf_counter()

In [40]:
t2-t1 # running time in seconds

37.625739788938006

For some unknown reason, we didn't succeeded in drawing ALS regret bound automatically via the above function. The regret bound in the presentation is generated by stocking first the single-run results manually and then drawing with the same commands.

In [64]:
t1 = time.perf_counter()
draw_regret_bound('ALS', nb_arms=3, nb_exp=10, nb_steps=5)
t2 = time.perf_counter()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ldr\\AppData\\Local\\Temp\\spark-b93eeb26-db00-49c2-9179-5d0770590ff7\\pyspark-47560876-bd60-4d20-9b91-35b70fcab783\\tmpva2u80gz'

In [ ]:
t2-t1